# Install and Import Packages

In [ ]:
## install required packages
!pip install swig
!pip install wrds
!pip install pyportfolioopt
## install finrl library
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

  Cloning https://github.com/AI4Finance-Foundation/FinRL.git to /tmp/pip-req-build-witdff13
  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/FinRL.git /tmp/pip-req-build-witdff13
  Resolved https://github.com/AI4Finance-Foundation/FinRL.git to commit 6156c42121c21fa153722c9bbf4deef3f8f79a79
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/AI4Finance-Foundation/ElegantRL.git to /tmp/pip-install-c066bo1d/elegantrl_79a00fd48cd94b12b5a209f38818129b
  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/ElegantRL.git /tmp/pip-install-c066bo1d/elegantrl_79a00fd48cd94b12b5a209f38818129b
  Resolved https://github.com/AI4Finance-Foundation/ElegantRL.git to commit 4340d2fabb884a8008b2d7a10cff4f76f627c7e9
  Preparing metadata (setup.py) ... done


In [ ]:
import pandas as pd
import numpy as np
import datetime
import yfinance as yf

from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl import config_tickers
from finrl.config import INDICATORS

import itertools

In [ ]:
TRAIN_START_DATE = '2009-01-01'
TRAIN_END_DATE = '2020-07-01'
TRADE_START_DATE = '2020-07-01'
TRADE_END_DATE = '2023-05-01'

In [ ]:
symbols = [
    'aapl',
    'msft',
    'meta',
    'ibm',
    'hd',
    'cat',
    'amzn',
    'intc',
    't',
    'v'
    'gs'
]

In [ ]:
df_raw = YahooDownloader(start_date = TRAIN_START_DATE,
                                end_date = TRADE_END_DATE,
                                ticker_list = symbols).fetch_data()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Shape of DataFrame:  (32958, 8)


In [ ]:
df_raw.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,date,open,high,low,close,volume,tic,day
0,2009-01-02,3.067143,3.251429,3.041429,2.737005,746015200,aapl,4
1,2009-01-02,2.567500,2.726500,2.553500,2.718000,145928000,amzn,4
2,2009-01-02,44.910000,46.980000,44.709999,30.712521,7117200,cat,4
3,2009-01-02,23.070000,24.190001,22.959999,16.435896,14902500,hd,4
4,2009-01-02,80.200768,83.738052,80.200768,48.733913,7905877,ibm,4


# Preprocess Data

In [ ]:
fe = FeatureEngineer(use_technical_indicator=True,
                     tech_indicator_list = INDICATORS,  ##indications like moving avg
                     use_vix=True,
                     use_turbulence=True,  ##Market unstable
                     user_defined_feature = False)

processed = fe.preprocess_data(df_raw)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[*********************100%***********************]  1 of 1 completed

Successfully added technical indicators
Shape of DataFrame:  (3604, 8)
Successfully added vix


Successfully added turbulence index


In [ ]:
processed

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,date,open,high,low,close,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2009-01-02,3.067143,3.251429,3.041429,2.737005,746015200,aapl,4,0.000000,2.958121,2.631401,100.000000,66.666667,100.000000,2.737005,2.737005,39.189999,0.00000
1,2009-01-02,2.567500,2.726500,2.553500,2.718000,145928000,amzn,4,0.000000,2.958121,2.631401,100.000000,66.666667,100.000000,2.718000,2.718000,39.189999,0.00000
2,2009-01-02,44.910000,46.980000,44.709999,30.712521,7117200,cat,4,0.000000,2.958121,2.631401,100.000000,66.666667,100.000000,30.712521,30.712521,39.189999,0.00000
3,2009-01-02,23.070000,24.190001,22.959999,16.435896,14902500,hd,4,0.000000,2.958121,2.631401,100.000000,66.666667,100.000000,16.435896,16.435896,39.189999,0.00000
4,2009-01-02,80.200768,83.738052,80.200768,48.733913,7905877,ibm,4,0.000000,2.958121,2.631401,100.000000,66.666667,100.000000,48.733913,48.733913,39.189999,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28827,2023-04-27,288.000000,294.709991,287.799988,284.353851,3133600,hd,3,0.856017,292.924510,275.919685,48.591903,15.318390,7.315064,281.480981,287.986492,17.030001,5.47556
28828,2023-04-27,126.370003,127.019997,125.459999,119.107475,3204900,ibm,3,-0.770919,124.905764,116.251004,44.911342,-38.511517,1.220107,119.846373,121.489781,17.030001,5.47556
28829,2023-04-27,28.750000,29.950001,28.510000,29.144180,60186200,intc,3,-0.018557,33.393176,28.197083,50.727395,-66.435367,13.482737,30.099059,28.330524,17.030001,5.47556
28830,2023-04-27,295.970001,305.200012,295.250000,301.261169,46462600,msft,3,5.080236,295.548252,272.647719,63.072938,228.756364,42.136850,280.462558,267.492234,17.030001,5.47556


In [ ]:
list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
combination = list(itertools.product(list_date,list_ticker))
combination

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[('2009-01-02', 'aapl'),
 ('2009-01-02', 'amzn'),
 ('2009-01-02', 'cat'),
 ('2009-01-02', 'hd'),
 ('2009-01-02', 'ibm'),
 ('2009-01-02', 'intc'),
 ('2009-01-02', 'msft'),
 ('2009-01-02', 't'),
 ('2009-01-03', 'aapl'),
 ('2009-01-03', 'amzn'),
 ('2009-01-03', 'cat'),
 ('2009-01-03', 'hd'),
 ('2009-01-03', 'ibm'),
 ('2009-01-03', 'intc'),
 ('2009-01-03', 'msft'),
 ('2009-01-03', 't'),
 ('2009-01-04', 'aapl'),
 ('2009-01-04', 'amzn'),
 ('2009-01-04', 'cat'),
 ('2009-01-04', 'hd'),
 ('2009-01-04', 'ibm'),
 ('2009-01-04', 'intc'),
 ('2009-01-04', 'msft'),
 ('2009-01-04', 't'),
 ('2009-01-05', 'aapl'),
 ('2009-01-05', 'amzn'),
 ('2009-01-05', 'cat'),
 ('2009-01-05', 'hd'),
 ('2009-01-05', 'ibm'),
 ('2009-01-05', 'intc'),
 ('2009-01-05', 'msft'),
 ('2009-01-05', 't'),
 ('2009-01-06', 'aapl'),
 ('2009-01-06', 'amzn'),
 ('2009-01-06', 'cat'),
 ('2009-01-06', 'hd'),
 ('2009-01-06', 'ibm'),
 ('2009-01-06', 'intc'),
 ('2009-01-06', 'msft'),
 ('2009-01-06', 't'),
 ('2009-01-07', 'aapl'),
 ('2009-01

In [ ]:
processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left") #dataframe consists of data of date and
# tic(stock) on processed data on left join where date column of combination merged on tic of processed
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])

processed_full = processed_full.fillna(0)

In [ ]:
processed_full

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2009-01-02,aapl,3.067143,3.251429,3.041429,2.737005,746015200.0,4.0,0.000000,2.958121,2.631401,100.000000,66.666667,100.000000,2.737005,2.737005,39.189999,0.00000
1,2009-01-02,amzn,2.567500,2.726500,2.553500,2.718000,145928000.0,4.0,0.000000,2.958121,2.631401,100.000000,66.666667,100.000000,2.718000,2.718000,39.189999,0.00000
2,2009-01-02,cat,44.910000,46.980000,44.709999,30.712521,7117200.0,4.0,0.000000,2.958121,2.631401,100.000000,66.666667,100.000000,30.712521,30.712521,39.189999,0.00000
3,2009-01-02,hd,23.070000,24.190001,22.959999,16.435896,14902500.0,4.0,0.000000,2.958121,2.631401,100.000000,66.666667,100.000000,16.435896,16.435896,39.189999,0.00000
4,2009-01-02,ibm,80.200768,83.738052,80.200768,48.733913,7905877.0,4.0,0.000000,2.958121,2.631401,100.000000,66.666667,100.000000,48.733913,48.733913,39.189999,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41827,2023-04-27,hd,288.000000,294.709991,287.799988,284.353851,3133600.0,3.0,0.856017,292.924510,275.919685,48.591903,15.318390,7.315064,281.480981,287.986492,17.030001,5.47556
41828,2023-04-27,ibm,126.370003,127.019997,125.459999,119.107475,3204900.0,3.0,-0.770919,124.905764,116.251004,44.911342,-38.511517,1.220107,119.846373,121.489781,17.030001,5.47556
41829,2023-04-27,intc,28.750000,29.950001,28.510000,29.144180,60186200.0,3.0,-0.018557,33.393176,28.197083,50.727395,-66.435367,13.482737,30.099059,28.330524,17.030001,5.47556
41830,2023-04-27,msft,295.970001,305.200012,295.250000,301.261169,46462600.0,3.0,5.080236,295.548252,272.647719,63.072938,228.756364,42.136850,280.462558,267.492234,17.030001,5.47556


# Save the Data

In [ ]:
# Split the data
train = data_split(processed_full, TRAIN_START_DATE,TRAIN_END_DATE)
trade = data_split(processed_full, TRADE_START_DATE,TRADE_END_DATE)
print(len(train))
print(len(trade))

23144
5688


In [ ]:
from google.colab import drive

drive.mount('/content/drive')

train_path = '/content/drive/My Drive/financial project/train.csv'
trade_path = '/content/drive/My Drive/financial project/trade.csv'

with open(train_path, 'w', encoding = 'utf-8-sig') as f:
  train.to_csv(f)

with open(trade_path, 'w', encoding = 'utf-8-sig') as f:
  trade.to_csv(f)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import drive

drive.mount('/content/drive')

train_path = '/content/drive/My Drive/train.csv' # Corrected file path
trade_path = '/content/drive/My Drive/trade.csv' # Corrected file path

with open(train_path, 'w', encoding = 'utf-8-sig') as f:
  train.to_csv(f)

with open(trade_path, 'w', encoding = 'utf-8-sig') as f:
  trade.to_csv(f)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Consider Joining the Newsleter

https://witty-motivator-1414.ck.page/acb393f729